In [1]:
!pip install -r ./../requirements.txt

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement 9pynvml==12.0.0 (from versions: none)

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for 9pynvml==12.0.0


In [ ]:
import torch
import time
from operations import OPS  # chứa các op giống trong DARTS
from codecarbon import EmissionsTracker

input_tensor = torch.randn(1, C, H, W)  # ví dụ: C=16, H=W=32

energy_lookup = {}

for op_name in PRIMITIVES:
    op = OPS[op_name](C, 1, True).cuda()
    op.eval()
    
    tracker = EmissionsTracker(measure_power_secs=1)
    tracker.start()
    
    with torch.no_grad():
        for _ in range(100):  # chạy nhiều lần để lấy trung bình
            _ = op(input_tensor.cuda())
    
    emissions = tracker.stop()
    energy_lookup[op_name] = emissions  # hoặc tracker.final_emissions_data


In [1]:
import torch
import time
from operations import OPS  # file bạn định nghĩa ops DARTS
from codecarbon import EmissionsTracker

# ⚙️ Bạn có thể điều chỉnh những giá trị này tùy theo thiết kế DARTS
C_list = [16, 32, 64]
HW_list = [32, 64]
PRIMITIVES = [
    'none',
    'max_pool_3x3',
    'avg_pool_3x3',
    'skip_connect',
    'sep_conv_3x3',
    'sep_conv_5x5',
    'dil_conv_3x3',
    'dil_conv_5x5',
    'snn_multistep_3x3',
    'snn_multistep_5x5',
    'conv_7x1_1x7',
    
]
# 📦 Kết quả sẽ được lưu vào đây
energy_lookup = {}

# 🔁 Duyệt qua từng tổ hợp C, H, W
for C in C_list:
    for H in HW_list:
        W = H
        input_tensor = torch.randn(1, C, H, W).cuda()

        for op_name in PRIMITIVES:
            key = (op_name, C, H, W)
            print(f"🔍 Benchmarking {key}")

            # Khởi tạo op
            op = OPS[op_name](C, 1, True).cuda()
            op.eval()

            tracker = EmissionsTracker(measure_power_secs=1, log_level="error")
            tracker.start()

            with torch.no_grad():
                # Warm-up
                for _ in range(10):
                    _ = op(input_tensor)

                # Benchmark
                start = time.time()
                for _ in range(100):
                    _ = op(input_tensor)
                torch.cuda.synchronize()
                end = time.time()

            emissions = tracker.stop()
            avg_time = (end - start) / 100

            # 🧠 Lưu kết quả
            energy_lookup[key] = {
                "emissions (kgCO2eq)": emissions,
                "avg_inference_time (s)": avg_time
            }

# 💾 Lưu file
import json
with open("energy_lookup.json", "w") as f:
    json.dump(energy_lookup, f, indent=4)

print("✅ Lookup table saved to energy_lookup.json")


ModuleNotFoundError: No module named 'codecarbon'

In [4]:
!pip install codecarbon

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.1 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
